# Pandas -csv・tsvファイルの読み込み、書き出し
---
csvやtsvなどの行列データを扱うのに便利なライブラリであるpandasについて解説。まずはデータの読み込みと書き込み。 


## 動作環境の確認（必要なライブラリインポート）

このNotebook環境に以下のライブラリがインストールされている必要あり。エラーになった場合は、適宜インストールが必要。

In [1]:
import pandas as pd

## CSVファイルの取りこみ

csvファイルの取込みは、pd.read_csv("ファイル名")。データフレーム型であるdfに格納される。データフレーム型は２次元の配列とイメージは同じ。それに加え、行名、列名という概念がある。
デフォルトでは1レコード目のcol1、col2、col3が列名として使用され、行名は自動的に0から昇順に振られる。

In [2]:
df1 = pd.read_csv("InputFile_01.csv")
df1

,col1,col2,col3
0,a,b,1
1,aa,bb,20
2,aaa,bbb,3


csvファイルの一行目からデータが収録されている場合（csvファイルに列名が収録されていない場合）、header=Noneを指定する。その場合、行名は0から昇順に振られる。

In [3]:
df2 = pd.read_csv('InputFile_01.csv',header=None)
df2

,0,1,2
0,col1,col2,col3
1,a,b,1
2,aa,bb,20
3,aaa,bbb,3


行名にcsvファイルのとある列を指定したい場合、index_col=列番号を指定する。列番号は0からカウント。

In [4]:
df3 = pd.read_csv('InputFile_01.csv',index_col=0)
df3

,col2,col3
col1,,
a,b,1
aa,bb,20
aaa,bbb,3


ファイルによっては一行目に制御系の情報(レコードサイズや業務日付)といった情報が収録されており、読み込みたくない場合がある。その場合はskiprows=行番号(2レコード目から取り込む場合は1を指定）

In [5]:
df4 = pd.read_csv('InputFile_01.csv',skiprows=1)
df4

,a,b,1
0,aa,bb,20
1,aaa,bbb,3


日本語も問題なく扱える(python 2系だとちょっと工夫が必要)。ただし、文字コード(windowsだとs-jis、Linuxやmacはutf-8)が違う場合はencodingオプションで指定する必要あり。  
InputFile_04.csvはs-jisのファイル。オプションを指定しない場合、文字化けする。

In [6]:
df5 = pd.read_csv('InputFile_04.csv')
df5

,���O,�l�i
0,���イ��,100
1,�֎q,120
2,�g�}�g,60


encoding='s-jis'を入れることで文字化けせずに読み込みできる。

In [7]:
df5 = pd.read_csv('InputFile_04.csv',encoding='s-jis')
df5

,名前,値段
0,きゅうり,100
1,茄子,120
2,トマト,60


InputFile_05.csvはutf-8のファイル。本環境はLinux上で動作するため、utf-8ならオプションがなくても文字化けしない。windows環境などでencodingを指定する場合は、encoding='utf-8'

In [8]:
df6 = pd.read_csv('InputFile_05.csv')
df6

,名前,値段
0,きゅうり,100
1,茄子,120
2,トマト,60


## tsvファイルの取込み

tsvファイル（区切り文字がtab)を読み込む場合は、read_table("ファイル名")。

In [9]:
df7 = pd.read_table('InputFile_02.tsv')
df7

,"col1,col2,col3"
0,"a,b,01"
1,"aa,bb,20"
2,"aaa,bbb,03"


## データフレームの各種情報の確認方法
行数を確認したい場合、len(データフレーム名)、行×列のサイズを確認したい場合は.shape。各列の型が知りたい場合は、.dtypes。

In [10]:
len(df7)

3

In [11]:
df7.shape

(3, 1)

In [12]:
df7.dtypes

col1,col2,col3    object
dtype: object

## ファイル読み込み時、列名をインプットファイルとは別に定義する
対象ファイルの1レコード目に列名がある場合は簡単だが、ない場合もある。列名を指定する場合はnames=["列名1","列名2","列名3"]

```c.csv
a,b,01
aa,bb,20
aaa,bbb,03
```
上記のように、データ部しかないcsvファイルを取り込むときは、names=リストで、列名のリストを渡す。

In [13]:
df8 = pd.read_csv('InputFile_03.csv',names=['col1','col2','col3'])
df8

,col1,col2,col3
0,a,b,1
1,aa,bb,20
2,aaa,bbb,3


## ファイル読み込み時、各列の型を指定する
型は自動的に判定されますが、01のようなコード値を数値型と判断して1にされてしまうことがある。型を明示的に指定して取り込みする場合はdtypes={"列名1":"型","列名2":"型","列名3":"型"}。文字列の場合は"S"(※)。整数型はint64(int32)、浮動小数型はfloat64(float32)。

col3を見ると頭0が消えている。DataFrameの型を確認したい場合、.dtypesを指定する。以下をみると、col3は数値型で取込まれている。

In [14]:
df8.dtypes

col1    object
col2    object
col3     int64
dtype: object

In [15]:
df9 = pd.read_csv('InputFile_03.csv',names=['col1','col2','col3'],dtype={'col3':'S'})
df9

,col1,col2,col3
0,a,b,01
1,aa,bb,20
2,aaa,bbb,03


## ファイルの書込み
ファイルの書込みはto_csv("ファイル名)にて行う。行名が不要な場合、index=Falseを指定する。また、デフォルトではcsv(カンマ区切り）のレイアウト。

In [16]:
df7.to_csv('OutputFile_01.csv',index=False)

タブ区切りなど、区切り文字を変えたい場合はsepを使う。

In [17]:
df7.to_csv('OutputFile_02.tsv',index=False,sep='	')

文字コードを指定する場合は、読み込み時同様、encodingで指定。

In [18]:
df5.to_csv('OutputFile_03.csv',encoding='s-jis')